In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import yaml

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

from env import SlidingEnv
import wrappers

pygame 2.5.2 (SDL 2.28.2, Python 3.8.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
2023-11-26 15:55:13.336177: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 15:55:13.838952: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/pandas/compat/nump

In [3]:
configs = {
    "total_timesteps": 1000000,

    "env_kwargs": {"w": 2, "h": 2, "shuffle_steps": 5},
    "n_envs": 16,
    "wrapper_class": wrappers.OneHotEncodingWrapper,

    "max_episode_steps": 100,
    "seed": 42,
}

In [4]:
gym.envs.register(
    id="SlidingEnv-v0",
    entry_point=SlidingEnv,
    max_episode_steps=configs["max_episode_steps"],
)

env = gym.make("SlidingEnv-v0", **configs["env_kwargs"])
env.reset()

(array([[3, 2],
        [0, 1]], dtype=int32),
 {})

In [5]:
env = make_vec_env(
    env_id="SlidingEnv-v0",
    seed=configs["seed"],
    wrapper_class=configs["wrapper_class"],
    env_kwargs=configs["env_kwargs"],
    n_envs=configs["n_envs"],
)

In [6]:
run_id = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
model = PPO(
    "MlpPolicy",
    env,
    # verbose=2,
    tensorboard_log=f"runs/{run_id}",
)

In [7]:
model.learn(
    total_timesteps=configs["total_timesteps"],
    progress_bar=True,
)

Output()

In [8]:
configs["run_id"] = run_id
with open(f"runs/{run_id}/configs.yaml", "w") as f:
    yaml.dump(configs, f)
model.save(f"runs/{run_id}/model")